In [52]:
import numpy as np
import csv
import pandas as pd
import networkx as nx
import os
import matplotlib.pyplot as plt

PATH = os.getcwd()

def read_file_from_csv(pathname, filename):
    # source = '%s%s/%s' % (PATH, pathname, filename)
    source = '%s/files/%s/%s' % (PATH, pathname, filename)
    df = None
    try:
        df = pd.read_csv(source, parse_dates=True, infer_datetime_format=True, na_filter=True, quoting=csv.QUOTE_ALL, thousands=',')
        print("read",len(df.index)," rows from", pathname + " " + filename)
    except:
        print("error opening the file %s" % source)
    finally:
        return df

In [31]:
def validUPC(u):
    upc = str(u).strip()
    upc = upc.replace(".0","")
    if not "UPC:" in upc:
        if not upc.isnumeric(): return ""
    else:
        upc = upc[4:]
    # let trim it
    upc = ("000000000000" + upc)[-12:]
    # start the validation algorithm
    check1 = 0
    for i in range(0,len(upc)-1,2):
        # print(upc[i])
        check1 += int(upc[i])
    check2 = check1 * 3
    check3 = 0
    for i in range(1,len(upc)-1,2):
        # print(upc[i])
        check3 += int(upc[i])
    check4 = check2 + check3
    rounded = round(check4/10)*10
    check5 = rounded - check4
    if check5 == int(upc[-1:]):
        # print(upc)
        return "UPC:" + upc
    else:
        # print("")
        return ""

validUPC("good")

''

How many invoices are in relationship with the same product?

In [28]:

df_catalog = read_file_from_csv("aws/dev-dashy-refined-datasets/archive/suppliers/birite/catalog", "BiRite Product List Full.csv")
# df_catalog["UPC"] = df_catalog["UPC"].astype(str)        
df_catalog["UPC"] = df_catalog["UPC"].fillna('')
df_catalog["UPC"] = df_catalog["UPC"].astype(str) 
df_catalog['UPC'] = df_catalog.UPC.str.strip()
df_catalog.loc[df_catalog['UPC'].isna(), 'UPC'] = ""
df_catalog.loc[df_catalog['UPC'] == 0, 'UPC'] = ""
df_catalog.loc[df_catalog['UPC'] == "0", 'UPC'] = ""
df_birite = read_file_from_csv("aws/dev-dashy-refined-datasets/datasets", "birite-merged.csv")


read 9189  rows from aws/dev-dashy-refined-datasets/archive/suppliers/birite/catalog BiRite Product List Full.csv
read 7243  rows from aws/dev-dashy-refined-datasets/datasets birite-merged.csv


In [29]:
df_catalog = df_catalog.sort_values(by=["UPC"], ascending=False)

df_catalog.head(5)
df_catalog.to_csv(PATH + "/files/" + "aws/dev-dashy-refined-datasets/archive/suppliers/birite/catalog/tmpcat.csv")

In [30]:
# df_catalog["SKU_copy"] = df_catalog["SKU"]
# df_birite["SKU_copy"] = df_birite["SKU"]
outer_merge = pd.merge(df_birite,df_catalog, on="SKU", how="outer", indicator=True)
outer_merge["URL"] = outer_merge["Product_URL"]
outer_merge["image_description"] = outer_merge["Product_Image"]
# outer_merge["UPC"] = outer_merge["UPC_y"]

outer_merge.loc[outer_merge['name'].isna(), 'name'] = outer_merge["Product_Title"]
outer_merge.loc[outer_merge['dataset'].isna(), 'dataset'] = "retailer"
outer_merge.loc[outer_merge['supplier'].isna(), 'supplier'] = "birite"
outer_merge["UPC_y"] = outer_merge["UPC_y"].fillna('')
outer_merge["UPC_y"] = outer_merge["UPC_y"].astype(str) 
outer_merge.loc[outer_merge['UPC_y'].isna(), 'UPC_y'] = ""
outer_merge.loc[outer_merge['UPC_y'] == 0, 'UPC_y'] = ""
outer_merge.loc[outer_merge['UPC_y'] == "0", 'UPC_y'] = ""
outer_merge["UPC"] = outer_merge["UPC_y"]
# outer_merge.head(5)

# outer_merge["SKU"]
outer_merge.to_csv(PATH + "/files/" + "aws/dev-dashy-refined-datasets/archive/suppliers/birite/catalog/tmp.csv")
outer_merge[outer_merge.UPC == ""]

,hash,dataset,item_code,supplier,name,brand,pack_count,unit_size,unit_measure,category,...,item_image,image_description,Product_URL,Product_Title,UPC_y,Product_Image,Product_Price,Extracted_Time,_merge,UPC
1,c9e025ff27095aa9a321227ba6b1070c,supplier,,birite,FLOUR ORGANIC CORN FINE GRIND YELLOW,GIUSTO,1.0,25,LB,FOOD,...,http://www.biritefoodservice.com/catalog/produ...,http://www.biritefoodservice.com/catalog/produ...,http://www.biritefoodservice.com/catalog/prod_...,FLOUR ORGANIC CORN FINE GRIND YELLOW,,http://www.biritefoodservice.com/catalog/produ...,NaN,50:35.0,both,
2,4fc7cbac3b65ff6403397e504c444fda,supplier,,birite,FLOUR ORGANIC BREAD ULTIMATE PERFORMER,GIUSTO,1.0,50,LB,FOOD,...,http://www.biritefoodservice.com/catalog/produ...,http://www.biritefoodservice.com/catalog/produ...,http://www.biritefoodservice.com/catalog/prod_...,FLOUR ORGANIC BREAD ULTIMATE PERFORMER,,http://www.biritefoodservice.com/catalog/produ...,NaN,50:34.9,both,
3,fd5bebe3766caa92a9c25540fec046ed,supplier,,birite,FLOUR ORGANIC AP BAKERS CHOICE UNBLCHD,GIUSTO,1.0,50,LB,FOOD,...,http://www.biritefoodservice.com/catalog/produ...,http://www.biritefoodservice.com/catalog/produ...,http://www.biritefoodservice.com/catalog/prod_...,FLOUR ORGANIC AP BAKERS CHOICE UNBLEACHD,,http://www.biritefoodservice.com/catalog/produ...,NaN,50:34.9,both,
4,36d13ee7fdadd501921572511bf11904,supplier,,birite,FLOUR ORGANIC AP ALL FAMILY 11% PROTIEN,GIUSTO,1.0,50,LB,FOOD,...,http://www.biritefoodservice.com/catalog/produ...,http://www.biritefoodservice.com/catalog/produ...,http://www.biritefoodservice.com/catalog/prod_...,FLOUR ORGANIC AP ALL FAMILY 11% PROTIEN,,http://www.biritefoodservice.com/catalog/produ...,NaN,50:34.8,both,
6,49bb3cc7d5e9a3a64ebf466cff2b2cea,supplier,,birite,FLOUR MASA CORN FLOUR MIX,MASECA,10.0,40.4,LB,FOOD,...,,NaN,NaN,NaN,,NaN,NaN,NaN,left_only,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11077,NaN,retailer,NaN,birite,"SPATULA OFFSET 9.75 X 1.5"" WOOD HANDLE",NaN,NaN,NaN,NaN,NaN,...,NaN,http://www.biritefoodservice.com/catalog/produ...,http://www.biritefoodservice.com/catalog/prod_...,"SPATULA OFFSET 9.75 X 1.5"" WOOD HANDLE",,http://www.biritefoodservice.com/catalog/produ...,NaN,54:01.0,right_only,
11078,NaN,retailer,NaN,birite,LID GRAY FOR 32 GAL BIN,NaN,NaN,NaN,NaN,NaN,...,NaN,http://www.biritefoodservice.com/catalog/produ...,http://www.biritefoodservice.com/catalog/prod_...,LID GRAY FOR 32 GAL BIN,,http://www.biritefoodservice.com/catalog/produ...,NaN,54:02.7,right_only,
11079,NaN,retailer,NaN,birite,SUGAR SYRUP GLUCOSE SMALL,NaN,NaN,NaN,NaN,NaN,...,NaN,http://www.biritefoodservice.com/catalog/produ...,http://www.biritefoodservice.com/catalog/prod_...,SUGAR SYRUP GLUCOSE SMALL,,http://www.biritefoodservice.com/catalog/produ...,NaN,08:19.8,right_only,
11080,NaN,retailer,NaN,birite,PAN BRAZIER HEAVY 24QT (OLD 4098),NaN,NaN,NaN,NaN,NaN,...,NaN,http://www.biritefoodservice.com/catalog/produ...,http://www.biritefoodservice.com/catalog/prod_...,PAN BRAZIER HEAVY 24QT (OLD 4098),,http://www.biritefoodservice.com/catalog/produ...,NaN,54:01.5,right_only,


In [92]:
import re
strn = 'Parfait Cup Insertin 6inches - 3.5" x 1.25 in.per case'
search = re.escape('per case')
# print (re.findall(r"\W" + search + r"\W", strn))
x = re.sub(r"(?<![a-zA-Z])"+search+ r"\b"," []", strn, flags=re.IGNORECASE)
print(x)




Parfait Cup Insertin 6inches - 3.5" x 1.25 in. []
